In [ ]:
__author__ = "Tim Cuddeback"

import yfpy
from yfpy.query import YahooFantasySportsQuery
import json
import openpyxl
from datetime import date

In [ ]:
# Function that pulls data from Yahoo's Fantasy API and returns a sorted list of tuples with (team_id,current points)
league_id = '12514' # put your real league id here
game_id = "399" # put the game id here (game id's reflect the type of sport and the year)
game_code = "nfl" # put the game code here
season = "2020" # put the year of the current fantasy season
auth_dir = '' # put the location where you are storing the client_id/secret
working_dir = '' # put the location where you want excel file to be saved
excel_file_name = 'Points.xlsx' # Name your file after you set up the headers
# Query yffpy to get data
yahoo_query = YahooFantasySportsQuery(auth_dir, league_id, game_id=game_id, game_code=game_code, offline=False)

# # Get standings data to get the points for the day
# standings_data = yahoo_query.get_league_standings()

data = yahoo_query.get_team_standings(1)
# .get_league_teams(self)
# .get_league_players(self)
# .get_league_draft_results(self)
# .get_league_matchups_by_week(self,chosen_week)
# .get_team_info(self,team_id)
# .get_team_standings(self,team_id) # has current "streak" within
# .get_player_stats_by_week(self, player_key, chosen_week="current")
data

# # Convert Standings Object to json
# parsed_json = (json.loads(str(data)))

# # Initialize a list that can store the data
# team_list = []
# # Iterate over the Standings json to get teamids and their respective points
# for team in parsed_json['teams']:
#     team_list.append((int(team['team']['team_id']),team['team']['team_standings']['points_for']))

# # Sort the team list by the team id
# team_list = sorted(team_list)
	


# Function that takes in a sorted list of tuples with (team_id,current points) and puts it in an excel sheet


# # open file and go to active sheet
# book = openpyxl.load_workbook(working_dir+excel_file_name)
# sheet = book.active

# # Create a list with the first column being the date and the following columns being the points for each respective team
# new_excel_row = []
# new_excel_row.append(date.today().strftime("%d/%m/%Y"))

# for team in team_list:
#     new_excel_row.append(team[1])

# # Add new row to sheet and save the file	
# sheet.append(new_excel_row)
# book.save(working_dir+excel_file_name)


# team_list_today = pull_data(season,league_id,game_id,game_code,auth_dir)
# write_to_sheet(working_dir,excel_file_name,team_list_today)